In [4]:
#general import
import os
import numpy as np
import pandas as pd

#nn import
from torch import device, load

#stats import
from scipy.stats import ttest_rel, wilcoxon
from statistics import median

#brain visualization import
from nilearn import regions, maskers, datasets, surface, plotting, image
from nilearn.plotting import plot_roi, plot_stat_map
from nilearn.maskers import NiftiMasker

#package import (for class) --> TO CHANGE
import sys
sys.path.append('/home/maelle/GitHub_repositories/cNeuromod_encoding_2020')  
import models

In [5]:
out_path = '/home/maelle/Results/figures/surface'
r2_test_model_path = '/home/maelle/Results/best_models'
r2_by_run_path = '/home/maelle/Results/best_models/predict_S4_runs'
MIST_path = '/home/maelle/DataBase/fMRI_parcellations/MIST_parcellation/Parcellations/MIST_ROI.nii.gz'
voxel_mask = '/home/maelle/GitHub_repositories/cNeuromod_encoding_2020/parcellation/STG_middle.nii.gz'
mist_roi_labels = '/home/maelle/DataBase/fMRI_parcellations/MIST_parcellation/Parcel_Information/MIST_ROI.csv'

label_pd = pd.read_csv(mist_roi_labels, sep=';')
print(label_pd.columns)

subs = ['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06']
scale = ['MIST_ROI', 'auditory_Voxels'] 
conv = 'conv4'
baseline = 'no_ft'
null_model = 'no_init'
diff_name = '{}-{}'.format(conv, baseline)

Index(['roi', 'label', 'name', 'size', 'symmetry', 'laterality', 'x', 'y', 'z',
       'neighbour', 'parent', 'overlap'],
      dtype='object')


In [ ]:

df_all = pd.DataFrame(columns = ['model_sub', 'scale', 'conv', ])
for sub in subs:
    sub_path = os.path.join(dir_path, dir_name+sub)
    data_sub = sub
    print(data_sub)
    for model in os.listdir(sub_path):
        model_path = os.path.join(sub_path, model)
        r2_array = np.load(model_path)
        r2max_array = np.max(r2_array, axis=1)
        
        model_sub = model[:len('sub-0')+1]
        scale = 'whole brain' if 'MIST_ROI' in model else 'stg'
        finetune_cond = model.find('_f_') 
        conv = model[finetune_cond+3:finetune_cond+3+len('conv')+1] if finetune_cond>-1 else 'no'
        
        model_data = {
            'model_sub':np.full(len(r2max_array), model_sub),
            'data_sub':np.full(len(r2max_array),data_sub), 
            'scale':np.full(len(r2max_array),scale), 
            'conv':np.full(len(r2max_array),conv), 
            'r²': r2max_array
                     }
        model_df = pd.DataFrame(model_data)
        df_all = pd.concat((df_all, model_df), axis='rows')

In [6]:
def sum_conditions(conditions) : 
    correct = True
    for condition in conditions:
        if condition == False:
            correct = False
            break
    return correct

In [7]:
def get_specific_file_path(path, *cond):
    specific_file_path = None
    for file in os.listdir(path):
        filepath = os.path.join(path, file)
        conditions = [(obj in file) if boolean else (obj not in file) for (obj, boolean) in cond]
        specific_file = sum_conditions(conditions)
        specific_file_path = filepath if specific_file else specific_file_path
    return specific_file_path

In [8]:
def df_runs_from_n_best_roi(data, n, columns_label = label_pd['label']):
    runs_df = pd.DataFrame(data, columns=columns_label)  
    median_runs_df = runs_df.median().sort_values(ascending=False)
    best_n_roi = list(median_runs_df.head(n).index)

    all_runs_df = runs_df.melt(var_name = 'roi', value_name='r2')
    best_runs = all_runs_df.loc[(all_runs_df['roi'].isin(best_n_roi))].copy()
    best_runs['subject'] = sub

    return (best_n_roi, best_runs)

In [9]:
def stats_between_pop(pop1, pop2, test = ttest_rel, threshold = 0.05, difference=False):
    sign_diff_rois = []
    for i, (pop1_roi, pop2_roi) in enumerate(zip(pop1, pop2)):
        if difference :
            pop1_roi = pop1_roi - pop2_roi
            pop2_roi = None
        stat, pvalue = test(pop1_roi, pop2_roi)
        if pvalue <= threshold :
            sign_diff_rois.append((i, pvalue))
    return sign_diff_rois

In [10]:
def best_rois_in_MIST_array(data_array, nb_occ=3):
    a = np.unique(np.argmax(data_array, axis=1), return_counts=True)
    nb_i = [(i, nb, label_pd['label'].iloc[i]) for i, nb in zip(a[0], a[1]) if nb>=nb_occ]
    return nb_i